In [ ]:
import json

import keras.saving
import numpy as np
from glob import glob
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelBinarizer # One-Hot Encoding을 위해 사용

In [ ]:
JOINT_NAME_TO_INDEX = {
    # Head & Face
    "코": 0, "왼쪽 눈": 1, "오른쪽 눈": 2,

    "왼쪽 귀": 7, "오른쪽 귀": 8,
    "왼쪽 어깨": 11, "오른쪽 어깨": 12,

    # Arms
    "왼쪽 팔꿈치": 13, "오른쪽 팔꿈치": 14,
    "왼쪽 손목": 15, "오른쪽 손목": 16,

    # Hands
    "왼쪽 엄지 손가락": 17,
    "오른쪽 엄지 손가락": 18,
    "왼쪽 중지 손가락": 19,
    "오른쪽 중지 손가락": 20,

    # Hips & Trunk
    "왼쪽 엉덩이": 23, "오른쪽 엉덩이": 24,

    # Legs
    "왼쪽 무릎": 25, "오른쪽 무릎": 26,
    "왼쪽 발목": 27, "오른쪽 발목": 28,
    "왼쪽 뒷꿈치": 29, "오른쪽 뒷꿈치": 30,
    "왼쪽 엄지 발가락": 31, "오른쪽 엄지 발가락": 32,
}

def normalize(raw_skeleton):
    normalized_skeleton = []
    for frame in raw_skeleton:
        P_ref = (frame[23] + frame[24]) / 2
        P_prime = frame - P_ref
        L = np.linalg.norm(frame[11] - frame[12])
        if L > 1e-6:
            P_double_prime = P_prime / L
        else:
            P_double_prime = P_prime  # 정규화하지 않음

        normalized_skeleton.append(P_double_prime)
    return np.array(normalized_skeleton)

def load_skeleton_from_json(json_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    pose_data = data["labelingInfo"][0]["pose"]["location"]

    skeleton_array = np.zeros((33, 2), dtype=np.float32)

    for k_name, v_coords in pose_data.items():

        if k_name in JOINT_NAME_TO_INDEX:
            index = JOINT_NAME_TO_INDEX[k_name]

            x = float(v_coords["x"])
            y = float(v_coords["y"])

            # 3. 해당 인덱스에 저장
            skeleton_array[index, 0] = x
            skeleton_array[index, 1] = y

    return skeleton_array

def load_and_preprocess_sequence_data(data_root):
    X_sequences = []
    Y_labels = []

    label_map = {
        '기본준비': 0,
        '앞굽이하고 아래막고 지르기': 1,
        '앞굽이하고 아래막기': 2,
        '앞굽이하고 지르기': 3,
        '앞서고 아래막기': 4,
        '앞서고 안막기': 5,
        '앞서고 얼굴막기': 6,
        '앞서고 지르기': 7,
        '앞차고 앞서고 지르기': 8
    }

    FRAME_TYPES = ['S01', 'M01', 'E01']

    grouped_data = {}

    json_files = glob(os.path.join(data_root, '**', '*.json'), recursive=True)

    for file_path in json_files:
        parts = file_path.split(os.sep)
        action_name = parts[-5]
        filename = parts[-1]

        name_parts = filename.split('-')
        attempt_id = '-'.join(name_parts[0:8])
        view_id = name_parts[-2]
        frame_type = name_parts[-1].split('.')[0]

        if frame_type in FRAME_TYPES:
            key = (action_name, attempt_id, view_id)

            if key not in grouped_data:
                grouped_data[key] = {}

            grouped_data[key][frame_type] = file_path

    for (action_name, attempt_id, view_id), paths in grouped_data.items():
        if all(ft in paths for ft in FRAME_TYPES):

            frame_data = []
            for ft in FRAME_TYPES:
                data = load_skeleton_from_json(paths[ft])
                frame_data.append(data)

            raw_sequence = np.stack(frame_data, axis=0)

            normalized_sequence = normalize(raw_sequence)

            X_sequences.append(normalized_sequence)
            Y_labels.append(label_map[action_name])

    return np.array(X_sequences), np.array(Y_labels), label_map


Code

Load, Normalize, Preprocess

In [7]:
ROOT_DIR = r"C:\Users\gony4\Desktop\태권도 태극 1장"

In [ ]:
X_sequences, Y_labels, label_map = load_and_preprocess_sequence_data(ROOT_DIR)

In [ ]:
unique, counts = np.unique(Y_labels, return_counts=True)
class_counts = dict(zip(unique, counts))

index_to_label = {v: k for k, v in label_map.items()}

print("--- 구분동작 별 샘플 개수 ---")
for index, count in class_counts.items():
    label_name = index_to_label.get(index, f"Unknown Index {index}")
    print(f"[{index}] {label_name}: {count} 개")

print(f"\n총 샘플 수: {len(Y_labels)} 개")

In [ ]:
X_FILE_NAME = 'X_taekwondo_sequence_3frame_33joint.npy'
Y_FILE_NAME = 'Y_taekwondo_labels.npy'

np.save(X_FILE_NAME, X_sequences)
np.save(Y_FILE_NAME, Y_labels)

print(f"X Data '{X_FILE_NAME}' saved")
print(f"Y Data '{Y_FILE_NAME}' saved")

In [9]:
X_FILE_NAME = 'data/X_taekwondo_sequence_3frame_33joint.npy'
Y_FILE_NAME = 'data/Y_taekwondo_labels.npy'

X_data = np.load(X_FILE_NAME)
Y_data = np.load(Y_FILE_NAME)

label_map = {
        '기본준비': 0,
        '앞굽이하고 아래막고 지르기': 1,
        '앞굽이하고 아래막기': 2,
        '앞굽이하고 지르기': 3,
        '앞서고 아래막기': 4,
        '앞서고 안막기': 5,
        '앞서고 얼굴막기': 6,
        '앞서고 지르기': 7,
        '앞차고 앞서고 지르기': 8
    }

print(f"Data Loaded. X_data shape: {X_data.shape}")

Data Loaded. X_data shape: (19983, 3, 33, 2)


LSTM

In [10]:
# Flatten the features
# (N, 3, 33, 2) -> (N, 3, 66)
N = X_data.shape[0]
X_processed = X_data.reshape(N, 3, 33 * 2)

N_CLASSES = len(label_map)
lb = LabelBinarizer()
Y_one_hot = lb.fit_transform(Y_data)

X_train, X_val, Y_train, Y_val = train_test_split(
    X_processed, Y_one_hot, test_size=0.2, random_state=42, stratify=Y_data # stratify로 클래스 비율 유지
)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(Y_data),
    y=Y_data
)
class_weight_dict = dict(enumerate(class_weights))

print(f"Final Learning Data Shape: {X_train.shape}")
print(f"Class weight dict: {class_weight_dict}")

Final Learning Data Shape: (15986, 3, 66)
Class weight dict: {0: np.float64(0.5374808359557814), 1: np.float64(2.7963895885810244), 2: np.float64(1.3613325158389535), 3: np.float64(0.842951151607188), 4: np.float64(0.6147102251753415), 5: np.float64(0.8312741794583801), 6: np.float64(1.4399048854301773), 7: np.float64(1.6325980392156862), 8: np.float64(1.380804311774461)}


In [11]:
TIMESTEPS = X_train.shape[1] # 3
INPUT_DIM = X_train.shape[2] # 66

model = Sequential([
    tf.keras.layers.InputLayer(shape=(TIMESTEPS, INPUT_DIM)),

    LSTM(units=128),

    Dropout(0.3),

    Dense(units=64, activation='relu'),

    Dense(N_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │        99,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 9)              │           585 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 108,681 (424.54 KB)

 Trainable params: 108,681 (424.54 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
EPOCHS = 50
BATCH_SIZE = 32

history = model.fit(
    X_train,
    Y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, Y_val),
    class_weight=class_weight_dict, # ⭐ 클래스 가중치 적용!
    verbose=1
)

print("\nModel Trained")

Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5144 - loss: 1.3286 - val_accuracy: 0.8461 - val_loss: 0.4495
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8491 - loss: 0.4114 - val_accuracy: 0.9022 - val_loss: 0.2732
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8850 - loss: 0.3098 - val_accuracy: 0.9057 - val_loss: 0.2668
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9001 - loss: 0.2734 - val_accuracy: 0.9194 - val_loss: 0.2255
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9154 - loss: 0.2349 - val_accuracy: 0.9470 - val_loss: 0.1565
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9294 - loss: 0.1986 - val_accuracy: 0.9417 - val_loss: 0.1664
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9371 - loss: 0.1737 - val_accuracy: 0.9422 - val_loss: 0.1548
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9338 - loss: 0.1803 - val_accuracy: 0.

In [13]:
loss, accuracy = model.evaluate(X_val, Y_val, verbose=0)

print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

Validation Loss: 0.0421
Validation Accuracy: 0.9860


In [14]:
from sklearn.metrics import classification_report

Y_pred_probs = model.predict(X_val)

Y_pred = np.argmax(Y_pred_probs, axis=1)

Y_true = np.argmax(Y_val, axis=1)

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [15]:
index_to_label = {v: k for k, v in label_map.items()}
sorted_class_names = [index_to_label[i] for i in sorted(index_to_label.keys())]

print("\n--- Classification Report ---")
print(classification_report(Y_true, Y_pred, target_names=sorted_class_names))


--- Classification Report ---
                precision    recall  f1-score   support

          기본준비       0.99      0.99      0.99       826
앞굽이하고 아래막고 지르기       0.97      0.99      0.98       159
    앞굽이하고 아래막기       0.99      0.98      0.99       326
     앞굽이하고 지르기       0.99      0.99      0.99       527
      앞서고 아래막기       0.99      0.98      0.99       723
       앞서고 안막기       0.99      0.97      0.98       534
      앞서고 얼굴막기       1.00      0.99      1.00       308
       앞서고 지르기       0.90      0.97      0.94       272
   앞차고 앞서고 지르기       1.00      1.00      1.00       322

      accuracy                           0.99      3997
     macro avg       0.98      0.99      0.98      3997
  weighted avg       0.99      0.99      0.99      3997



In [16]:
import keras

# HDF5
MODEL_SAVE_PATH = 'taekwondo_lstm_model.h5'
keras.saving.save_model(model, MODEL_SAVE_PATH)

print(f"Model saved: '{MODEL_SAVE_PATH}'")

Model saved: 'taekwondo_lstm_model.h5'


In [ ]:
tfjs_target_dir = 'tfjs_output'

tfjs.converters.save_keras_model(model, tfjs_target_dir)